In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from cvxpy.atoms.affine.wraps import psd_wrap
from read_data import *
from sklearn.model_selection import train_test_split
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%       MGT - 418         %%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Convex Optimization - Project 2          %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%             2021-2022 Fall                    %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%      Learning the Kernel Function             %%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [25]:
def kernel_learning(K1, K2, K3, y_tr, rho):
    """
    Kernel learning for soft margin SVM. 
    Implementation of problem (5)
    Use cvxpy.atoms.affine.psd_wrap for each G(\hat K^l) matrix when it appear in the constraints and in the objective
    """
    
    #Y = np.outer(y_tr,y_tr.T)
    
    lambda_ = cp.Variable(len(y_tr))
    z = cp.Variable(1)
    
    c = np.trace(K1+K2+K3)
    
    
   
    cons = []
    K = [K1,K2,K3]
    for k_i in K : 
        cons.append(z * np.trace(k_i) >= 1/ (2 * rho) * cp.quad_form(lambda_, psd_wrap(np.diag(y_tr) * k_i * np.diag(y_tr))))
    cons.append(lambda_<= 1)
    cons.append(lambda_>=0)
    cons.append(lambda_.T @ y_tr == 0)
    
    obj = cp.Maximize(cp.vstack([lambda_.T]) @ np.ones(len(y_tr)) -c*z)
    
    
    print("ole")
    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.MOSEK)

    
    mu_opt1 = cons[0].dual_value
    mu_opt2 = cons[1].dual_value
    mu_opt3 = cons[2].dual_value

    
    b_opt = cons[5].dual_value
    return mu_opt1, mu_opt2, mu_opt3, lambda_.value, b_opt

In [ ]:
def svm_fit(kernel, y_tr, rho):
    """
    Dual of soft-margin SVM problem (2)
    Use cvxpy.atoms.affine.psd_wrap for each G(\hat K^l) matrix when it appear in the constraints and in the objective
    """
    n_tr = len(y_tr)
    G =  ...
    lambda_ = cp.Variable(n_tr)
    dual_obj = cp.Maximize(... cp.quad_form(lambda_, psd_wrap(G)))
    cons = []
    ...
    prob = cp.Problem(dual_obj, cons)
    prob.solve(solver=cp.MOSEK)
    lambda_opt = lambda_.value
    b_opt =  ...
    return lambda_opt, b_opt

In [73]:
def svm_predict(kernel, y_tr, y_te, lambda_opt, b_opt, rho):
    """
    Predict function for kernel SVM. 
    See lecture slide 183.
    """
    n_te = len(y_te)
    n_tr = len(y_tr)
    good = 0
    for i in range(n_te): 
        tot = 0
        for j in range(n_tr): 
            tot = tot + lambda_opt[j]*y_tr[j]*kernel[i,j] + b_opt
            
        if np.sign((1/rho)*tot) == y_te[i] : 
            good = good + 1
  
    acc = good/n_te
    return acc


In [3]:
a = prepare_ionosphere_dataset()
X_train, X_test, y_train, y_test = train_test_split(a[0],a[1],test_size = 0.2, random_state=0)

def k_1(x,y, arg): 
    k_1 = (1.0 + np.dot(x.T,y))**int(arg)
    return k_1

def k_2(x,y,arg) : 
    k_2 = np.exp(-np.dot((x-y).T,(x-y))/2*0.5)
    return k_2

def k_3(x,y,arg): 
    k_3 = np.dot(x.T,y)
    return k_3

def K_creator(X_train, k_func,arg): 
    K = np.zeros((X_train.shape[0],X_train.shape[0]))
    for i in range(X_train.shape[0]) : 
        for j in range(X_train.shape[0]): 
            K[i,j] = k_func(X_train[i,:],X_train[j,:],arg)
    return K
K_func = [k_1,k_2, k_3]
args = [2.0,0.5,None]
K = [None]*3
for i in range(3):
    K[i] = K_creator(X_train,K_func[i],args[i])
    

In [55]:
result = kernel_learning(K[0],K[1],K[2],y_train,2)
result

print( result[0][0])
print( result[1][0])
print( result[2][0])
print( len(result[3]))


ole
0.10935620189555241
241.6629189479908
3.602580529048555e-05
280


In [75]:
kernel = result[0][0]*K[0] + K[1]*result[1][0]+K[2]*result[2][0]
kernel

svm_predict(kernel, y_train,y_test,result[3],result[4], 0.5)

0.5774647887323944

In [75]:
acc_opt_kernel = []    
acc_poly_kernel = []    
acc_gauss_kernel = []    
acc_linear_kernel = []    
rho = 0.01
data, labels = prepare_ionosphere_dataset()
for iters in range(100): 
    ## Please do not change the random seed.
    np.random.seed(iters)
    ### Training-test split
    msk = np.random.rand(data_normalized.shape[0]) <=...
    x_tr = data[...]
    x_te = data[...]
    y_tr = labels[...]
    y_te = labels[...]
 
    n_tr = y_tr.shape[0]
    n_te = y_te.shape[0]
    n_tr = x_tr.shape[0]
    n_te = x_te.shape[0]
    
    x_all = np.vstack([x_tr, x_te])
    n_all = x_all.shape[0]

    ## Prepare the initial choice of kernels 
    # It is recommended to prepare the kernels for all the training and the test data
    # Then, the kernel size will be (n_tr + n_te)x(n_tr + n_te).
    # Use only the training block (like K1[0:n_tr, 0:n_tr] ) to learn the classifier 
    # (for the functions svm_fit and kernel_learning).
    # When predicting you may use the whole kernel as it is. 
    K1 = ...
    K2 = ...
    K3 = ...

    mu_opt1, mu_opt2, mu_opt3, lambda_opt, b_opt = kernel_learning(...)
    opt_kernel = ...
    acc_opt_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_poly_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_gauss_kernel.append(svm_predict(...))
    
    lambda_opt, b_opt = svm_fit(...)
    acc_linear_kernel.append(svm_predict(...)
    print('Iteration-->' + str(iters))
print('Average dual accuracy with optimal kernel is ' + str(np.mean(acc_opt_kernel)))
print('Average dual accuracy with polynomial kernel is ' + str(np.mean(acc_poly_kernel)))
print('Average dual accuracy with gaussian kernel is ' + str(np.mean(acc_gauss_kernel)))
print('Average dual accuracy with linear kernel is ' + str(np.mean(acc_linear_kernel)))